In [3]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9120/860599/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1581611378&Signature=XRXJdQkSswHY5iMTnjVSXiaJaaVY7zrXqaOliII2EaggoXI3pDpfrY5DyIJhiYaob1s1ZE6PVCR0H3%2BzSfb6ryuWmEO2n9fqsAGN7wRxGn6u7xW8Y7YC258RVjvTVKLvp2Ls5ODf4XexGqMTqteAhZ9nZIOPIeUTCiqN3sqAPr65JRgghDoYBx4eAnvXqMi7vVUx%2B5wMk86qQV1Qv64H4%2BVBpn3%2BZ2BDJOxcrc%2FK2W7lxs6VuSppzjbMH9MSJ9pqZMF2mI%2FfuBqDwf1ZwcaawK2GG7Gt20XZrCyuaVctcRT2GGShvIWpbjtj4q4G9Txo6Ry2o2N4YTosVuuP1pdROA%3D%3D&response-content-disposition=attachment%3B+filename%3Dhome-credit-default-risk.zip" -O "home-credit-default-risk.zip" -c

--2020-02-10 16:29:58--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9120/860599/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1581611378&Signature=XRXJdQkSswHY5iMTnjVSXiaJaaVY7zrXqaOliII2EaggoXI3pDpfrY5DyIJhiYaob1s1ZE6PVCR0H3%2BzSfb6ryuWmEO2n9fqsAGN7wRxGn6u7xW8Y7YC258RVjvTVKLvp2Ls5ODf4XexGqMTqteAhZ9nZIOPIeUTCiqN3sqAPr65JRgghDoYBx4eAnvXqMi7vVUx%2B5wMk86qQV1Qv64H4%2BVBpn3%2BZ2BDJOxcrc%2FK2W7lxs6VuSppzjbMH9MSJ9pqZMF2mI%2FfuBqDwf1ZwcaawK2GG7Gt20XZrCyuaVctcRT2GGShvIWpbjtj4q4G9Txo6Ry2o2N4YTosVuuP1pdROA%3D%3D&response-content-disposition=attachment%3B+filename%3Dhome-credit-default-risk.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 721616255 (688M) [application/zip]
Saving to: ‘home-credit-default-risk.zip’

home-credit-

In [4]:
!unzip 'home-credit-default-risk.zip'

Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [5]:
!pip install lightgbm

In [0]:
import pandas as pd


In [0]:
import gc
import pickle
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def final_fun_1(t_df):
  # code taken from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
  def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

  
  
  def one_hot(df):
    original_col = list(df.columns)
    c = [c for c in df.columns if df[c].dtype == 'object']
    df = pd.get_dummies(df, columns= c, dummy_na= True)
    new_col = [c for c in df.columns if c not in original_col]
    return df, new_col
 
  df = reduce_mem_usage(pd.read_csv('application_train.csv'))
  test_df = reduce_mem_usage(t_df)
  df = df.append(test_df).reset_index()

  df = df[df['CODE_GENDER'] != 'XNA']
  df = df[df['AMT_GOODS_PRICE'].notnull()]
  df = df[df['NAME_INCOME_TYPE'] != 'Maternity leave']
  df = df[df['DAYS_LAST_PHONE_CHANGE'].notnull()]
  df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

  df['DAYS_BIRTH / DAYS_EMPLOYED'] = df['DAYS_BIRTH'] / df['DAYS_EMPLOYED']

  df['AMT_CREDIT / AMT_INCOME_TOTAL'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
  df['CNT_FAM_MEMBERS / AMT_INCOME_TOTAL'] = df['CNT_FAM_MEMBERS'] / df['AMT_INCOME_TOTAL']
  df['AMT_ANNUITY / AMT_INCOME_TOTAL'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
    
  df['AMT_CREDIT / AMT_ANNUITY'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
  df['ANNUITY_LENGTH / DAYS_EMPLOYED'] = df['AMT_CREDIT / AMT_ANNUITY'] / df['DAYS_EMPLOYED']
  df['CNT_CHILDREN / CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
  
  df['AMT_CREDIT / AMT_GOODS_PRICE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
  df['AMT_CREDIT / AMT_GOODS_PRICE'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']
  df['DAYS_REGISTRATION / DAYS_ID_PUBLISH'] = df['DAYS_REGISTRATION'] / df['DAYS_ID_PUBLISH']
  df['DAYS_BIRTH / DAYS_REGISTRATION'] = df['DAYS_BIRTH'] / df['DAYS_REGISTRATION']
  df['DOC_SUM'] = df['FLAG_DOCUMENT_2'] + df['FLAG_DOCUMENT_3'] + df['FLAG_DOCUMENT_4'] + df['FLAG_DOCUMENT_5'] + df['FLAG_DOCUMENT_6'] + df['FLAG_DOCUMENT_7'] + df['FLAG_DOCUMENT_8'] + df['FLAG_DOCUMENT_9'] + df['FLAG_DOCUMENT_10'] +df['FLAG_DOCUMENT_11'] + df['FLAG_DOCUMENT_12'] + df['FLAG_DOCUMENT_13'] + df['FLAG_DOCUMENT_14'] + df['FLAG_DOCUMENT_15'] + df['FLAG_DOCUMENT_16'] + df['FLAG_DOCUMENT_17'] + df['FLAG_DOCUMENT_18'] + df['FLAG_DOCUMENT_19'] + df['FLAG_DOCUMENT_20'] + df['FLAG_DOCUMENT_21']

  df['NAN_AMT_ANNUITY'] = 1.0*np.isnan(df['AMT_ANNUITY'])
  df['AGE_FINISH'] = df['DAYS_BIRTH']*(-1.0/365) + (df['AMT_CREDIT']/df['AMT_ANNUITY']) *(1.0/12) #how old when finish
    
  docs = [data for data in df.columns if 'FLAG_DOC' in data]
  live = [data for data in df.columns if ('FLAG_' in data) & ('FLAG_DOC' not in data) & ('_FLAG_' not in data)]
  df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
  df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
  df['AMT_INCOME_TOTAL / CNT_CHILDREN'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
  org_type = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
  df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(org_type)
  df['AMT_INCOME_TOTAL / NEW_INC_BY_ORG'] = df['AMT_INCOME_TOTAL'] / df['NEW_INC_BY_ORG']
  df['OWN_CAR_AGE / DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
  df['OWN_CAR_AGE / DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
  df['DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
  df['DAYS_LAST_PHONE_CHANGE / DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
     
  for f in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
      df[f], c = pd.factorize(df[f])

  cat_col = [i for i in df.columns if df[i].dtype == 'object']
  df = pd.get_dummies(df, columns= cat_col)

  df= df.drop(['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],axis=1)
  #print(df.shape) 
  del t_df,test_df
  gc.collect()

  def b_new(buro):
    bureau_balance = reduce_mem_usage(pd.read_csv('bureau_balance.csv'))
    bureau_balance, col1 = one_hot(bureau_balance)
    bureau, col2= one_hot(buro)

    bureau_balance_agg = {'MONTHS_BALANCE': ['size']}
    for c in col1:
      bureau_balance_agg[c] = ['mean']
    aggregate = bureau_balance.groupby('SK_ID_BUREAU').agg(bureau_balance_agg)
    l=[]
    for i in aggregate.columns.tolist():
      l.append(i[0] + "_" + i[1].upper())
    
    aggregate.columns = pd.Index(l)
    bureau = bureau.join(aggregate, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del aggregate
    gc.collect()
    
    numerical_agg = {'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'DAYS_CREDIT': ['mean', 'var'],'DAYS_CREDIT_UPDATE': ['mean'],'CREDIT_DAY_OVERDUE': ['mean'],
        'DAYS_CREDIT_ENDDATE': ['mean'],'CNT_CREDIT_PROLONG': ['sum'],'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_ANNUITY': ['max', 'mean'],'AMT_CREDIT_SUM': ['mean', 'sum']
      }
    categorical_agg = {}
    for c in col2:
      categorical_agg[c] = ['mean']
    for c in col1:
      categorical_agg[c + "_MEAN"] = ['mean']
    
    b1 = bureau.groupby('SK_ID_CURR').agg({**numerical_agg, **categorical_agg})
    l=[]
    for i in b1.columns.tolist():
      l.append('BU_'+i[0]+'_'+i[1].upper())
    b1.columns = pd.Index(l)
     
    A = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    A_agg = A.groupby('SK_ID_CURR').agg(numerical_agg)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('A_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    b1 = b1.join(A_agg, how='left', on='SK_ID_CURR')
    del A,A_agg
    gc.collect()

    C = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    C_agg = C.groupby('SK_ID_CURR').agg(numerical_agg)
    l=[]
    for i in C_agg.columns.tolist():
      l.append('C_'+i[0]+'_'+i[1].upper())
    C_agg.columns = pd.Index(l)
    b1 = b1.join(C_agg, how='left', on='SK_ID_CURR')
    del C, C_agg
    gc.collect()
  
    #print(b1.shape)
    del bureau
    gc.collect()
    return(b1)

 
  def pos_cash(pos):
    pos_ch, col = one_hot(pos)
    aggregate = {'SK_DPD_DEF': ['max', 'mean'],'SK_DPD': ['max', 'mean'],
        'MONTHS_BALANCE': ['max', 'mean', 'size']
        
    }
    for c in col:
        aggregate[c] = ['mean']
    
    A_agg = pos_ch.groupby('SK_ID_CURR').agg(aggregate)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('POS_'+i[0] + '_' + i[1].upper())
    A_agg.columns = pd.Index(l)
    A_agg['POS_no_COUNT'] = pos_ch.groupby('SK_ID_CURR').size()

    value=pos_ch.groupby('SK_ID_CURR')['MONTHS_BALANCE'].min()
    A_agg['MIN_VALUES']=pos_ch['SK_ID_CURR'].map(value)
    A_agg['MULTIPLIER']=1.00-pos_ch['MONTHS_BALANCE']/A_agg['MIN_VALUES']
    #print(A_agg.shape)
    del pos_ch
    gc.collect()
    return A_agg

  def credit_card(cre):
    credit_card, col = one_hot(cre)
    credit_card.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    credit_card_agg = credit_card.groupby('SK_ID_CURR').agg(['max', 'mean', 'sum', 'var'])
    l=[]
    for i in credit_card_agg.columns.tolist():
      l.append('CR_'+i[0]+'_'+i[1].upper())
    credit_card_agg.columns = pd.Index(l)
    # Count credit card lines
    credit_card_agg['CR_no_COUNT'] = credit_card.groupby('SK_ID_CURR').size()
    #print(credit_card_agg.shape)
    del credit_card
    gc.collect()
    return credit_card_agg

  def previous_app(prev):
    p_app, p_app_cols = one_hot(prev)
    p_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    p_app['APP_CREDIT_PERC'] = p_app['AMT_APPLICATION'] / p_app['AMT_CREDIT']
    num_agg = {'AMT_DOWN_PAYMENT': ['max', 'mean'],'DAYS_DECISION': ['max', 'mean'],
                        'AMT_GOODS_PRICE': ['max', 'mean'],'AMT_ANNUITY': ['max', 'mean'],'HOUR_APPR_PROCESS_START': ['max', 'mean'],
        'RATE_DOWN_PAYMENT': ['max', 'mean'],'AMT_APPLICATION': ['max', 'mean'],
        'AMT_CREDIT': ['max', 'mean'],'APP_CREDIT_PERC': ['max', 'mean'],'CNT_PAYMENT': ['mean', 'sum'] 
          
    }
    cat_agg = {}
    for cat in p_app_cols:
        cat_agg[cat] = ['mean']
    
    P1 = p_app.groupby('SK_ID_CURR').agg({**num_agg, **cat_agg})
    l=[]
    for i in P1.columns.tolist():
      l.append('PR_'+i[0]+'_'+i[1].upper())
    P1.columns = pd.Index(l)
    A = p_app[p_app['NAME_CONTRACT_STATUS_Approved'] == 1]
    A_agg = A.groupby('SK_ID_CURR').agg(num_agg)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('APP_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    P1 = P1.join(A_agg, how='left', on='SK_ID_CURR')
    R = p_app[p_app['NAME_CONTRACT_STATUS_Refused'] == 1]
    R_agg = R.groupby('SK_ID_CURR').agg(num_agg)
    l=[]
    for i in R_agg.columns.tolist():
      l.append('REF_'+i[0]+'_'+i[1].upper())
    
    R_agg.columns = pd.Index(l)
    P1 = P1.join(R_agg, how='left', on='SK_ID_CURR')
    del R, R_agg, A, A_agg, p_app
    gc.collect()
    #print(P1.shape)
    return P1

  def inst_pay(ins):
    A, col = one_hot(ins)
    
    aggregate = {'days_late_on_payment': ['max','mean','min','sum'],'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'NUM_INSTALMENT_VERSION': ['nunique'],'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'amount_extra_paid': ['max','mean','min','sum'],'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']     
    }
    for c in col:
        aggregate[c] = ['mean']
    A_agg = A.groupby('SK_ID_CURR').agg(aggregate)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('INS_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    A_agg['INSTAL_no_COUNT'] = A.groupby('SK_ID_CURR').size()
    del A
    gc.collect()
    #print(A_agg.shape)
    return A_agg


  b=pd.read_csv('bureau.csv')
     
  new = b_new(reduce_mem_usage(b))
  df = df.join(new, how='left', on='SK_ID_CURR')
  del new
  gc.collect()

    
  pr = pd.read_csv('previous_application.csv')

  frame = previous_app(reduce_mem_usage(pr))
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-365].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()
  
  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-182].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-92].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-31].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-8].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_week')
  del frame
  gc.collect()

  c = reduce_mem_usage(pd.read_csv('POS_CASH_balance.csv'))
  frame = pos_cash(c)
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()
        
  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-12].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()
  
  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-6].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-3].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-1].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()


  #i=pd.read_csv('inst_feats.csv')
  i=pd.read_csv('installments_payments.csv')
  i['DAYS_ENTRY_PAYMENT'].fillna(100, inplace=True)
  i['AMT_PAYMENT'].fillna(0.0, inplace=True)
  i['days_late_on_payment']=i.DAYS_ENTRY_PAYMENT-i.DAYS_INSTALMENT
  i['amount_extra_paid']=i.AMT_PAYMENT-i.AMT_INSTALMENT
  i = reduce_mem_usage(i)
  frame = inst_pay(i)
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-365].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-182].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-92].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-31].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-8].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_week')
  del frame
  gc.collect()

  cr=pd.read_csv('credit_card_balance.csv')
  frame = credit_card(reduce_mem_usage(cr))
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-12].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)     
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-6].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-3].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)     
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-1].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  with open('98_model1.pickle', 'rb') as handle:
    m1 = pickle.load(handle)
  with open('98_model2.pickle', 'rb') as handle:
    m2 = pickle.load(handle)
  with open('98_model3.pickle', 'rb') as handle:
    m3 = pickle.load(handle)
  with open('98_model4.pickle', 'rb') as handle:
    m4 = pickle.load(handle)
  with open('98_model5.pickle', 'rb') as handle:
    m5 = pickle.load(handle)
  with open('98_model6.pickle', 'rb') as handle:
    m6 = pickle.load(handle)
  with open('98_model7.pickle', 'rb') as handle:
    m7 = pickle.load(handle)
  with open('98_model8.pickle', 'rb') as handle:
    m8 = pickle.load(handle)
  with open('98_model9.pickle', 'rb') as handle:
    m9 = pickle.load(handle)
  with open('98_model10.pickle', 'rb') as handle:
    m10 = pickle.load(handle)  

  train_df = df[df['TARGET'].notnull()]
  test_df = df[df['TARGET'].isnull()]
  #print("test shape: {}".format(test_df.shape))
  del df
  gc.collect()

  feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
  out = np.zeros(test_df.shape[0])
  
  out += m1.predict_proba(test_df[feats], num_iteration=m1.best_iteration_)[:, 1] / 10
  #print('model1 finish')
  out += m2.predict_proba(test_df[feats], num_iteration=m2.best_iteration_)[:, 1] / 10
  #print('model2 finish')
  out += m3.predict_proba(test_df[feats], num_iteration=m3.best_iteration_)[:, 1] / 10
  #print('model3 finish')
  out += m4.predict_proba(test_df[feats], num_iteration=m4.best_iteration_)[:, 1] / 10
  #print('model4 finish')
  out += m5.predict_proba(test_df[feats], num_iteration=m5.best_iteration_)[:, 1] / 10
  #print('model5 finish')
  out += m6.predict_proba(test_df[feats], num_iteration=m6.best_iteration_)[:, 1] / 10
  #print('model6 finish')
  out += m7.predict_proba(test_df[feats], num_iteration=m7.best_iteration_)[:, 1] / 10
  #print('model7 finish')
  out += m8.predict_proba(test_df[feats], num_iteration=m8.best_iteration_)[:, 1] / 10
  #print('model8 finish')
  out += m9.predict_proba(test_df[feats], num_iteration=m9.best_iteration_)[:, 1] / 10
  #print('model9 finish')
  out += m10.predict_proba(test_df[feats], num_iteration=m10.best_iteration_)[:, 1] / 10
  #print('model10 finish')

  test_df['TARGET'] = out
  test_df[['SK_ID_CURR', 'TARGET']].to_csv('test1_model.csv', index= False)
  return test_df


In [0]:
import gc
import pickle
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning)

def final_fun_2(t_df,y):
  # code taken from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
  def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

  
  
  def one_hot(df):
    original_col = list(df.columns)
    c = [c for c in df.columns if df[c].dtype == 'object']
    df = pd.get_dummies(df, columns= c, dummy_na= True)
    new_col = [c for c in df.columns if c not in original_col]
    return df, new_col
 
  #df = reduce_mem_usage(pd.read_csv('application_train.csv'))
  df = reduce_mem_usage(t_df)
  #df = df.reset_index()
 
  df = df[df['CODE_GENDER'] != 'XNA']
  df = df[df['AMT_GOODS_PRICE'].notnull()]
  df = df[df['NAME_INCOME_TYPE'] != 'Maternity leave']
  df = df[df['DAYS_LAST_PHONE_CHANGE'].notnull()]
  df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

  df['DAYS_BIRTH / DAYS_EMPLOYED'] = df['DAYS_BIRTH'] / df['DAYS_EMPLOYED']

  df['AMT_CREDIT / AMT_INCOME_TOTAL'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
  df['CNT_FAM_MEMBERS / AMT_INCOME_TOTAL'] = df['CNT_FAM_MEMBERS'] / df['AMT_INCOME_TOTAL']
  df['AMT_ANNUITY / AMT_INCOME_TOTAL'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
    
  df['AMT_CREDIT / AMT_ANNUITY'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
  df['ANNUITY_LENGTH / DAYS_EMPLOYED'] = df['AMT_CREDIT / AMT_ANNUITY'] / df['DAYS_EMPLOYED']
  df['CNT_CHILDREN / CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
  
  df['AMT_CREDIT / AMT_GOODS_PRICE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
  df['AMT_CREDIT / AMT_GOODS_PRICE'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']
  df['DAYS_REGISTRATION / DAYS_ID_PUBLISH'] = df['DAYS_REGISTRATION'] / df['DAYS_ID_PUBLISH']
  df['DAYS_BIRTH / DAYS_REGISTRATION'] = df['DAYS_BIRTH'] / df['DAYS_REGISTRATION']
  df['DOC_SUM'] = df['FLAG_DOCUMENT_2'] + df['FLAG_DOCUMENT_3'] + df['FLAG_DOCUMENT_4'] + df['FLAG_DOCUMENT_5'] + df['FLAG_DOCUMENT_6'] + df['FLAG_DOCUMENT_7'] + df['FLAG_DOCUMENT_8'] + df['FLAG_DOCUMENT_9'] + df['FLAG_DOCUMENT_10'] +df['FLAG_DOCUMENT_11'] + df['FLAG_DOCUMENT_12'] + df['FLAG_DOCUMENT_13'] + df['FLAG_DOCUMENT_14'] + df['FLAG_DOCUMENT_15'] + df['FLAG_DOCUMENT_16'] + df['FLAG_DOCUMENT_17'] + df['FLAG_DOCUMENT_18'] + df['FLAG_DOCUMENT_19'] + df['FLAG_DOCUMENT_20'] + df['FLAG_DOCUMENT_21']

  df['NAN_AMT_ANNUITY'] = 1.0*np.isnan(df['AMT_ANNUITY'])
  df['AGE_FINISH'] = df['DAYS_BIRTH']*(-1.0/365) + (df['AMT_CREDIT']/df['AMT_ANNUITY']) *(1.0/12) #how old when finish
    
  docs = [data for data in df.columns if 'FLAG_DOC' in data]
  live = [data for data in df.columns if ('FLAG_' in data) & ('FLAG_DOC' not in data) & ('_FLAG_' not in data)]
  df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
  df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
  df['AMT_INCOME_TOTAL / CNT_CHILDREN'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
  org_type = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
  df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(org_type)
  df['AMT_INCOME_TOTAL / NEW_INC_BY_ORG'] = df['AMT_INCOME_TOTAL'] / df['NEW_INC_BY_ORG']
  df['OWN_CAR_AGE / DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
  df['OWN_CAR_AGE / DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
  df['DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
  df['DAYS_LAST_PHONE_CHANGE / DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
     
  for f in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
      df[f], c = pd.factorize(df[f])

  cat_col = [i for i in df.columns if df[i].dtype == 'object']
  df = pd.get_dummies(df, columns= cat_col)

  df= df.drop(['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],axis=1)
  #print(df.shape) 
  del t_df
  gc.collect()

  def b_new(buro):
    bureau_balance = reduce_mem_usage(pd.read_csv('bureau_balance.csv'))
    bureau_balance, col1 = one_hot(bureau_balance)
    bureau, col2= one_hot(buro)

    bureau_balance_agg = {'MONTHS_BALANCE': ['size']}
    for c in col1:
      bureau_balance_agg[c] = ['mean']
    aggregate = bureau_balance.groupby('SK_ID_BUREAU').agg(bureau_balance_agg)
    l=[]
    for i in aggregate.columns.tolist():
      l.append(i[0] + "_" + i[1].upper())
    
    aggregate.columns = pd.Index(l)
    bureau = bureau.join(aggregate, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del aggregate
    gc.collect()
    
    numerical_agg = {'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'DAYS_CREDIT': ['mean', 'var'],'DAYS_CREDIT_UPDATE': ['mean'],'CREDIT_DAY_OVERDUE': ['mean'],
        'DAYS_CREDIT_ENDDATE': ['mean'],'CNT_CREDIT_PROLONG': ['sum'],'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_ANNUITY': ['max', 'mean'],'AMT_CREDIT_SUM': ['mean', 'sum']
      }
    categorical_agg = {}
    for c in col2:
      categorical_agg[c] = ['mean']
    for c in col1:
      categorical_agg[c + "_MEAN"] = ['mean']
    
    b1 = bureau.groupby('SK_ID_CURR').agg({**numerical_agg, **categorical_agg})
    l=[]
    for i in b1.columns.tolist():
      l.append('BU_'+i[0]+'_'+i[1].upper())
    b1.columns = pd.Index(l)
     
    A = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    A_agg = A.groupby('SK_ID_CURR').agg(numerical_agg)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('A_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    b1 = b1.join(A_agg, how='left', on='SK_ID_CURR')
    del A,A_agg
    gc.collect()

    C = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    C_agg = C.groupby('SK_ID_CURR').agg(numerical_agg)
    l=[]
    for i in C_agg.columns.tolist():
      l.append('C_'+i[0]+'_'+i[1].upper())
    C_agg.columns = pd.Index(l)
    b1 = b1.join(C_agg, how='left', on='SK_ID_CURR')
    del C, C_agg
    gc.collect()
  
    #print(b1.shape)
    del bureau
    gc.collect()
    return(b1)

 
  def pos_cash(pos):
    pos_ch, col = one_hot(pos)
    aggregate = {'SK_DPD_DEF': ['max', 'mean'],'SK_DPD': ['max', 'mean'],
        'MONTHS_BALANCE': ['max', 'mean', 'size']
        
    }
    for c in col:
        aggregate[c] = ['mean']
    
    A_agg = pos_ch.groupby('SK_ID_CURR').agg(aggregate)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('POS_'+i[0] + '_' + i[1].upper())
    A_agg.columns = pd.Index(l)
    A_agg['POS_no_COUNT'] = pos_ch.groupby('SK_ID_CURR').size()

    value=pos_ch.groupby('SK_ID_CURR')['MONTHS_BALANCE'].min()
    A_agg['MIN_VALUES']=pos_ch['SK_ID_CURR'].map(value)
    A_agg['MULTIPLIER']=1.00-pos_ch['MONTHS_BALANCE']/A_agg['MIN_VALUES']
    #print(A_agg.shape)
    del pos_ch
    gc.collect()
    return A_agg

  def credit_card(cre):
    credit_card, col = one_hot(cre)
    credit_card.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    credit_card_agg = credit_card.groupby('SK_ID_CURR').agg(['max', 'mean', 'sum', 'var'])
    l=[]
    for i in credit_card_agg.columns.tolist():
      l.append('CR_'+i[0]+'_'+i[1].upper())
    credit_card_agg.columns = pd.Index(l)
    # Count credit card lines
    credit_card_agg['CR_no_COUNT'] = credit_card.groupby('SK_ID_CURR').size()
    #print(credit_card_agg.shape)
    del credit_card
    gc.collect()
    return credit_card_agg

  def previous_app(prev):
    p_app, p_app_cols = one_hot(prev)
    p_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    p_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    p_app['APP_CREDIT_PERC'] = p_app['AMT_APPLICATION'] / p_app['AMT_CREDIT']
    num_agg = {'AMT_DOWN_PAYMENT': ['max', 'mean'],'DAYS_DECISION': ['max', 'mean'],
                        'AMT_GOODS_PRICE': ['max', 'mean'],'AMT_ANNUITY': ['max', 'mean'],'HOUR_APPR_PROCESS_START': ['max', 'mean'],
        'RATE_DOWN_PAYMENT': ['max', 'mean'],'AMT_APPLICATION': ['max', 'mean'],
        'AMT_CREDIT': ['max', 'mean'],'APP_CREDIT_PERC': ['max', 'mean'],'CNT_PAYMENT': ['mean', 'sum'] 
          
    }
    cat_agg = {}
    for cat in p_app_cols:
        cat_agg[cat] = ['mean']
    
    P1 = p_app.groupby('SK_ID_CURR').agg({**num_agg, **cat_agg})
    l=[]
    for i in P1.columns.tolist():
      l.append('PR_'+i[0]+'_'+i[1].upper())
    P1.columns = pd.Index(l)
    A = p_app[p_app['NAME_CONTRACT_STATUS_Approved'] == 1]
    A_agg = A.groupby('SK_ID_CURR').agg(num_agg)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('APP_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    P1 = P1.join(A_agg, how='left', on='SK_ID_CURR')
    R = p_app[p_app['NAME_CONTRACT_STATUS_Refused'] == 1]
    R_agg = R.groupby('SK_ID_CURR').agg(num_agg)
    l=[]
    for i in R_agg.columns.tolist():
      l.append('REF_'+i[0]+'_'+i[1].upper())
    
    R_agg.columns = pd.Index(l)
    P1 = P1.join(R_agg, how='left', on='SK_ID_CURR')
    del R, R_agg, A, A_agg, p_app
    gc.collect()
    #print(P1.shape)
    return P1

  def inst_pay(ins):
    A, col = one_hot(ins)
    
    aggregate = {'days_late_on_payment': ['max','mean','min','sum'],'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'NUM_INSTALMENT_VERSION': ['nunique'],'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'amount_extra_paid': ['max','mean','min','sum'],'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']     
    }
    for c in col:
        aggregate[c] = ['mean']
    A_agg = A.groupby('SK_ID_CURR').agg(aggregate)
    l=[]
    for i in A_agg.columns.tolist():
      l.append('INS_'+i[0]+'_'+i[1].upper())
    A_agg.columns = pd.Index(l)
    A_agg['INSTAL_no_COUNT'] = A.groupby('SK_ID_CURR').size()
    del A
    gc.collect()
    #print(A_agg.shape)
    return A_agg


  b=pd.read_csv('bureau.csv')
     
  new = b_new(reduce_mem_usage(b))
  df = df.join(new, how='left', on='SK_ID_CURR')
  del new
  gc.collect()

    
  pr = pd.read_csv('previous_application.csv')

  frame = previous_app(reduce_mem_usage(pr))
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-365].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()
  
  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-182].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-92].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-31].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  frame = reduce_mem_usage(pr[pr.DAYS_DECISION  >=-8].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = previous_app(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_week')
  del frame
  gc.collect()

  c = reduce_mem_usage(pd.read_csv('POS_CASH_balance.csv'))
  frame = pos_cash(c)
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()
        
  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-12].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()
  
  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-6].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-3].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(c[c.MONTHS_BALANCE >=-1].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = pos_cash(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()


  #i=pd.read_csv('inst_feats.csv')
  i=pd.read_csv('installments_payments.csv')
  i['DAYS_ENTRY_PAYMENT'].fillna(100, inplace=True)
  i['AMT_PAYMENT'].fillna(0.0, inplace=True)
  i['days_late_on_payment']=i.DAYS_ENTRY_PAYMENT-i.DAYS_INSTALMENT
  i['amount_extra_paid']=i.AMT_PAYMENT-i.AMT_INSTALMENT
  i = reduce_mem_usage(i)
  frame = inst_pay(i)
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-365].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-182].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-92].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-31].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  frame = reduce_mem_usage(i[i.DAYS_INSTALMENT  >=-8].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = inst_pay(frame)       
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_week')
  del frame
  gc.collect()

  cr=pd.read_csv('credit_card_balance.csv')
  frame = credit_card(reduce_mem_usage(cr))
  df = df.join(frame, how='left', on='SK_ID_CURR')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-12].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)     
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_year')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-6].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_halfyear')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-3].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)     
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_quarter')
  del frame
  gc.collect()

  frame = reduce_mem_usage(cr[cr.MONTHS_BALANCE >=-1].reset_index())
  frame.drop(['index'],axis=1,inplace=True)
  frame = credit_card(frame)      
  df = df.join(frame, how='left', on='SK_ID_CURR',rsuffix='_month')
  del frame
  gc.collect()

  with open('98_model1.pickle', 'rb') as handle:
    m1 = pickle.load(handle)
  with open('98_model2.pickle', 'rb') as handle:
    m2 = pickle.load(handle)
  with open('98_model3.pickle', 'rb') as handle:
    m3 = pickle.load(handle)
  with open('98_model4.pickle', 'rb') as handle:
    m4 = pickle.load(handle)
  with open('98_model5.pickle', 'rb') as handle:
    m5 = pickle.load(handle)
  with open('98_model6.pickle', 'rb') as handle:
    m6 = pickle.load(handle)
  with open('98_model7.pickle', 'rb') as handle:
    m7 = pickle.load(handle)
  with open('98_model8.pickle', 'rb') as handle:
    m8 = pickle.load(handle)
  with open('98_model9.pickle', 'rb') as handle:
    m9 = pickle.load(handle)
  with open('98_model10.pickle', 'rb') as handle:
    m10 = pickle.load(handle)  
  
  train_df = df[:250000]
  y=train_df['TARGET']
  #test_df = df[df['TARGET'].isnull()]
  #print("train shape: {}".format(train_df.shape))
  del df
  gc.collect()

  feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
  out = np.zeros(train_df.shape[0])
  out += m1.predict_proba(train_df[feats], num_iteration=m1.best_iteration_)[:,1] / 10
  #print('model1 finish')
  out += m2.predict_proba(train_df[feats], num_iteration=m2.best_iteration_)[:, 1] / 10
  #print('model2 finish')
  out += m3.predict_proba(train_df[feats], num_iteration=m3.best_iteration_)[:, 1] / 10
  #print('model3 finish')
  out += m4.predict_proba(train_df[feats], num_iteration=m4.best_iteration_)[:, 1] / 10
  #print('model4 finish')
  out += m5.predict_proba(train_df[feats], num_iteration=m5.best_iteration_)[:, 1] / 10
  #print('model5 finish')
  out += m6.predict_proba(train_df[feats], num_iteration=m6.best_iteration_)[:, 1] / 10
  #print('model6 finish')
  out += m7.predict_proba(train_df[feats], num_iteration=m7.best_iteration_)[:, 1] / 10
  #print('model7 finish')
  out += m8.predict_proba(train_df[feats], num_iteration=m8.best_iteration_)[:, 1] / 10
  #print('model8 finish')
  out += m9.predict_proba(train_df[feats], num_iteration=m9.best_iteration_)[:, 1] / 10
  #print('model9 finish')
  out += m10.predict_proba(train_df[feats], num_iteration=m10.best_iteration_)[:, 1] / 10
  #print('model10 finish')
  
  return roc_auc_score(y,out)


In [32]:
test_df=pd.read_csv('application_test.csv')
test_df=test_df[:1]
test=final_fun_1(test_df)
test[['SK_ID_CURR', 'TARGET']]

,SK_ID_CURR,TARGET
307511,100001,0.035038


In [33]:
train_df=pd.read_csv('application_train.csv')
train_df=train_df.reset_index()
y=train_df['TARGET']
print(final_fun_2(train_df,y))

0.7836583223570655
